# GLANSIS REFERENCE CLEANER & BULK UPLOADER
**Description:** The following scripts will help you to clean and bulk upload references

**Installing Libraries:** To run this script, there are several necessary packages that need to be installed. Below is the quick and easy way to install the necessary pacakges to run this code. You only need to run it the first time you run this script. After that, the packages will be installed in your system. For that reason, I have the code commented out because there should be no reason to run any other time. IF this is your first time, remove the '#' to uncomment the second line ('pip install requirement.text'). Make sure the requirements.txt is in the main folder. 

*Be Aware: This is not the 'proper' way to initalize a script. If you find yourself running multiple scripts for different project and are frequently installing new packages, you should create a virtual environment. There is plenty of resources online explaining how to so. 


In [ ]:
# install Libaries - keep this commented after firest use
# pip install -r requirements.txt


# PART 2: Bulk Uploader

## 1. Run Widget to Start Bulk Uploading References

In [1]:
# DO NOT EDIT
import pandas as pd              # Managing dataframes
import os                        # Setting working directores
import tkinter as tk             # Creates GUI
from tkinter import filedialog   # Creates file dialog pop-up
from tkinter import messagebox   # Creates message box for errors
from bs4 import BeautifulSoup    # HTML parsing
import time                      # Inputs pauses to let webpage load
from selenium import webdriver                                     # automate web browser interaction
from selenium.webdriver.common.keys import Keys                    # automate keyboard actions
from selenium.webdriver.common.by import By                        # find elements by html id on webpage
from selenium.webdriver.support.ui import Select                   # automate dropdown selection
from selenium.webdriver.support.ui import WebDriverWait            # command driver to wait
from selenium.webdriver.support import expected_conditions as EC   # command driver to wait until loaded
from selenium.webdriver.chrome.options import Options              # option to make webdriver not visible
from selenium.common.exceptions import WebDriverException          # Deal with exptions in webdriver


# Command to open Excel with references
def open_excel_file():
    
    # Make ref a global variable to use other functions
    global ref

    # Open a file dialog to select an Excel file
    file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx;*.xls")])

    # Read the Excel file into a DataFrame using pandas, applying the specified data types
    ref = pd.read_excel(file_path, dtype = str)

    # Replace NaN to blanks
    ref.fillna('', inplace = True)
    
    # Check if ref exists
    if ref.empty:
        
        # If ref is empty, show an error message
        ref_success_label.config(text = "Error - Select correct file with references.", fg = "red")
        
    else:
        
        # If ref exists, proceed with saving
        ref_success_label.config(text = "Success!", fg = "green")


# Command to select PDF folder
def select_pdf_folder():
    
    # Make pdfs_folder_path a global variable
    global pdfs_folder_path
    
    # Select PDF folder
    pdfs_folder_path = filedialog.askdirectory(title="Select a Folder")
    
    # Check if pdf_folder_path exists
    if not pdfs_folder_path:
        
        # If species pdfs_folder_path does not exist, show an error message
        pdf_folder_success_label.config(text = "Error - Select PDF folder.", fg="red")
        
    else:
        
        # If df is empty, show an error message
        pdf_folder_success_label.config(text = "Success!", fg="green")
    

# Command to open webpage to login
def open_webdriver():
    
    global driver
    
    # Set up Chrome webdriver options
    option = webdriver.ChromeOptions()
    
    # Add experimental option to detach the browser window
    option.add_experimental_option("detach", True)
    
    # Initialize Chrome webdriver with the specified options
    driver = webdriver.Chrome(options = option)
    
    # Navigate to the specified URL
    driver.get('https://nas.er.usgs.gov/DataEntry/References/Default.aspx')
    

# Command to bulk upload references
def run_selenium_script():
    
    global ref 
    
    # Create empty list for new RefNum
    results = []
    
    # Create empty list for error messages
    error_messages = []

    # Input references and pdfs into NAS database 
    for index, row in ref.iterrows():  

        try:

            # Click on 'New' Button to create new reference
            new_button = driver.find_element(By.ID, 'ContentPlaceHolder1_New')
            new_button.click()

            time.sleep(0.5)

            # Find dropdown elements on the web page and select information - TYPE
            type_dropdown = Select(driver.find_element(By.ID, 'ContentPlaceHolder1_type'))
            type_dropdown.select_by_visible_text(str(row["Type"]))

            time.sleep(0.5)

            # Find input elements on the web page and fill them with data - AUTHOR
            author_input = driver.find_element(By.ID, 'ContentPlaceHolder1_author')
            author_input.send_keys(str(row["Author"]))

            # Find input elements on the web page and fill them with data - YEAR
            year_input = driver.find_element(By.ID, 'ContentPlaceHolder1_date')
            year_input.send_keys(str(row["Year"]))

            # Find dataframe input elements on the web page and fill them with data - TITLE
            time.sleep(0.5)

            driver.switch_to.frame(0)   # switch focus to the title iframe
            title_input = driver.find_element(By.CLASS_NAME, 'cke_show_borders')   # locate 
            title_input.click()
            title_input.send_keys(str(row["Title"]))   # submit input elements
            driver.switch_to.default_content()   # Switch back to the main content

            # Find input elements on the web page and fill them with data - JOURNAL NAME
            journal_input = driver.find_element(By.ID, 'ContentPlaceHolder1_journal')
            journal_input.send_keys(str(row["Journal Name"]))

            # Find input elements on the web page and fill them with data - VOLUME
            vol_input = driver.find_element(By.ID, 'ContentPlaceHolder1_vol')
            vol_input.send_keys(str(row["Volume"]))

            # Find input elements on the web page and fill them with data - ISSUE
            issue_input = driver.find_element(By.ID, 'ContentPlaceHolder1_issue')
            issue_input.send_keys(str(row["Issue"]))

            # Find input elements on the web page and fill them with data - PAGES
            page_input = driver.find_element(By.ID, 'ContentPlaceHolder1_pages')
            page_input.send_keys(str(row["Pages"]))

            # Find input elements on the web page and fill them with data - PAGES
            page_input = driver.find_element(By.ID, 'ContentPlaceHolder1_URL')
            page_input.send_keys(str(row["URL"]))

            # Find dropdown elements on the web page and select information - SPECIMEN DATA
            specimen_dropdown = Select(driver.find_element(By.ID, 'ContentPlaceHolder1_entered'))
            specimen_dropdown.select_by_visible_text(str(row["Specimen Data Entered"]))

            # Find dropdown elements on the web page and select information - IMPACT DATA
            impact_dropdown = Select(driver.find_element(By.ID, 'ContentPlaceHolder1_impacts'))
            impact_dropdown.select_by_visible_text(str(row["Impacts Data Entered"]))

            time.sleep(0.5)

            # Find dropdown elements on the web page and select information - LOCATION
            location_dropdown = Select(driver.find_element(By.ID, 'ContentPlaceHolder1_LocationDDL'))
            location_dropdown.select_by_visible_text(str(row["Location"]))

            # Find input elements on the web page and fill them with data - KEYWORDS
            keyword_input = driver.find_element(By.ID, 'ContentPlaceHolder1_key_words')
            keyword_input.send_keys(str(row["Keywords"]))

            # Find input elements on the web page and fill them with data - ABSTRACT
            time.sleep(0.5)

            driver.switch_to.frame(1)   # switch focus to the title iframe
            abstract_input = driver.find_element(By.CLASS_NAME, 'cke_show_borders')   # locate
            abstract_input.click()
            abstract_input.send_keys(str(row["Abstract"]))   # submit input elements
            driver.switch_to.default_content()   # Switch back to the main content

            # Find input elements on the web page and fill them with data - DOI
            doi_input = driver.find_element(By.ID, 'ContentPlaceHolder1_DOI')
            doi_input.send_keys(str(row["DOI"]))

            # Submit the form
            submit_button = driver.find_element(By.ID, 'ContentPlaceHolder1_Submit')
            # submit_button.click()

            # Accept alert to add PDF
            driver.switch_to.alert.accept()

            # Switch back to default content
            driver.switch_to.default_content()


            # Get html script from web page and find RefNum
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "ContentPlaceHolder1_refnum")))
            html = driver.page_source
            soup = BeautifulSoup(html,'html.parser')
            refnum = soup.find("span", {"id": "ContentPlaceHolder1_refnum"}).get_text()

            # append refnum to result list for later
            results.append(refnum)

            # Rename File
            old_name = row["PDF Name"]
            new_name = refnum + '.pdf'
            os.rename(os.path.join(pdfs_folder_path, old_name), os.path.join(pdfs_folder_path, new_name))

            # Choose file
            time.sleep(1)
            choose_button = driver.find_element(By.ID, 'ContentPlaceHolder1_FileInput')
            choose_button.send_keys(os.path.join(pdfs_folder_path, new_name))

            # Submit button
            submit_button = driver.find_element(By.ID, 'ContentPlaceHolder1_Submit')
            submit_button.click()

            # Continue button
            continue_button = driver.find_element(By.ID, 'ContentPlaceHolder1_Button1')
            continue_button.click()


        except Exception as e:
            
            # Append refnum to result list for later
            results.append('NA')
            
            # Create error message
            error_text = f"Error row {index + 1}: {str(e).splitlines()[0]}\nAuthors: {row['Author']} \nYear: {row['Year']} \nTitle: {row['Title']}\n"

            # Append to error message list
            error_messages.append(str(error_text))
            
            # Skip to the next iteration of the loop
            continue

    # Add Reference Number to excel sheet
    ref['RefNum'] = results 
    
    # Create message box
    def show_error_messages(messages):

        # Create a new top-level window
        error_window = tk.Toplevel()
        error_window.title("Error in Upload")

        # Add a scrollbar
        scrollbar = tk.Scrollbar(error_window)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        # Create a text widget to display the error messages
        text = tk.Text(error_window, wrap=tk.WORD, yscrollcommand=scrollbar.set, font=("Arial", 9))
        text.pack(expand=True, fill=tk.BOTH)
        scrollbar.config(command=text.yview)

        # Insert error messages into the text widget
        for msg in messages:
            text.insert(tk.END, msg + "\n")

        # Disable editing in the text widget
        text.config(state=tk.DISABLED)

        # Set the dimensions of the window
        error_window.geometry("400x400")  # Change dimensions as needed

        # Make the window modal (disables interaction with other windows)
        error_window.transient()
        error_window.grab_set()

        # Run the window's event loop
        error_window.mainloop()

    # Keyword error message     
    if error_messages:

        # Run function for error messages
        show_error_messages(error_messages)


# Command to export Excel
def export_excel():
    
    global ref 
    
    #reorder columsn
    new_column_order = ['RefNum', "Type", "Author", "Year", "Title", "Journal Name", "Volume", "Issue", "Pages", "URL", "Keywords", "Abstract", "DOI", "PDF Name"]
    ref = ref[new_column_order]
    
    # Select location to save Excel
    file_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])
    ref.to_excel(file_path, index=False)

    # Open the Excel file with the default application
    os.system(file_path)


# Create a tkinter window
window = tk.Tk()
window.title("GLANSIS Bulk Uploader")

# Header paragraph
header_text = "Bulk Uploader: The following widget will upload references into the NAS database."
header_label = tk.Label(window, text = header_text, wraplength = 450, justify = "left")
header_label.grid(row = 0, padx = 10, pady = (10, 20))


# Description of label button
button_text = "Click buttons below to import reference information for bulk upload"
button_label = tk.Label(window, text = button_text,  wraplength = 450, justify = "left")
button_label.grid(row = 1, padx = 10, pady = (10, 0), sticky = "w")

# Import reference button
ref_import_button = tk.Button(window, text = "Select Excel File", command = open_excel_file, width = 15, height = 1)
ref_import_button.grid(row = 2, column = 0, padx = 10, pady = 10, sticky = "w")
ref_success_label = tk.Label(window, text = "")
ref_success_label.grid(row = 2, padx = 150, pady = 10, sticky = "w")

# Select PDF folder button
pdf_button = tk.Button(window, text = "Select PDF Folder", command = select_pdf_folder, width = 15, height = 1)
pdf_button.grid(row = 3, column = 0, padx = 10, pady = 10, sticky = "w")
pdf_folder_success_label = tk.Label(window, text = "")
pdf_folder_success_label.grid(row = 3, padx = 150, pady = 10, sticky = "w")


# Description of label button
open_button_text = "Click buttons below to open NAS data entry website - you will need to use personal login credentials"
open_button_label = tk.Label(window, text = open_button_text,  wraplength = 450, justify = "left")
open_button_label.grid(row = 4, padx = 10, pady = (10, 0), sticky = "w")

# Open webdriver button
open_button = tk.Button(window, text = "Open New Window", command = open_webdriver, width = 15, height = 1)
open_button.grid(row = 5, padx = 10, pady = 10, sticky = "w")


# Description of label button
upload_button_text = "Click buttons below to upload references into NAS"
upload_button_label = tk.Label(window, text = upload_button_text,  wraplength = 450, justify = "left")
upload_button_label.grid(row = 6, padx = 10, pady = (10, 0), sticky = "w")

# Bulk upload button
upload_button = tk.Button(window, text = "Upload References", command = run_selenium_script, width = 15, height = 1)
upload_button.grid(row = 7, padx = 10, pady = (10, 20), sticky = "w")


# Description of label button
export_button_text = "Click buttons below to export new reference information"
export_button_label = tk.Label(window, text = export_button_text,  wraplength = 450, justify = "left")
export_button_label.grid(row = 8, padx = 10, pady = (10, 0), sticky = "w")

# Export excel button
excel_export_button = tk.Button(window, text = "Export Excel", command = export_excel, width = 15, height = 1)
excel_export_button.grid(row = 9, padx = 10, pady = (10, 20), sticky = "w")

# Quality Assurance Note
qa_text = checklist_text = """** Double-check to make sure everything ran properly:

1. Check your 'PDFs' folder - you should be able to see the PDFs that have been renamed with their new references number.
2. Check new reference numbers have been appended. """

qa_label = tk.Label(window, text = qa_text, wraplength = 450, justify = "left")
qa_label.grid(row = 11, padx = 10, pady = (10, 20), sticky = 'w')


# Run the tkinter event loop
window.mainloop()
